# TikTok Claims — Exploratory Data Analysis

**Owner:** Katherine Ygbuhay  
**Updated:** September 2025  
**Stage:** 02 

**Goal:**  
Perform structured exploratory data analysis (EDA) to profile the dataset, assess quality, and surface distributional patterns or imbalances that may affect modeling.  

**Contents:**  
- Dataset structure and variable types  
- Missing values and data quality checks  
- Standardization of categorical fields  
- Frequency and balance checks across claim and author ban status  
- Distributional review of engagement counts (skewness, outliers, variance)

## Dataset Overview and Schema

In [1]:
# Core packages
import pandas as pd
import numpy as np

# Plotly renderer setup for consistent inline visuals in JupyterLab
import plotly.io as pio
pio.renderers.default = "jupyterlab"  # fallback: "png" if running outside Lab

# Improve readability of numeric outputs
pd.options.display.float_format = '{:.3f}'.format

In [2]:
# Resolve the case-study root so paths work from any launch directory
from pathlib import Path

def find_case_root(start: Path | None = None) -> Path:
    p = start or Path.cwd()
    for q in [p, *p.parents]:
        if (q / "notebooks").exists() and (q / "data").exists():
            return q
    return p  # fallback

CASE_ROOT = find_case_root()
DATA_FILE = CASE_ROOT / "data" / "raw" / "tiktok_dataset.csv"
assert DATA_FILE.exists(), f"Missing data file: {DATA_FILE}"

In [3]:
# Load dataset into dataframe
df = pd.read_csv(DATA_FILE)

In [4]:
# --- Basic overview: shape, columns, and a small preview --------------------
# Shape: quick sense of dataset size (rows, columns)
print("Shape:", df.shape)

# Columns: ordered list of field names to orient yourself
print("Columns:", list(df.columns))

# Preview: first 10 rows to eyeball values, formats, and obvious anomalies
display(df.head(10))

Shape: (19382, 12)
Columns: ['#', 'claim_status', 'video_id', 'video_duration_sec', 'video_transcription_text', 'verified_status', 'author_ban_status', 'video_view_count', 'video_like_count', 'video_share_count', 'video_download_count', 'video_comment_count']


,#,claim_status,video_id,video_duration_sec,video_transcription_text,verified_status,author_ban_status,video_view_count,video_like_count,video_share_count,video_download_count,video_comment_count
0,1,claim,7017666017,59,someone shared with me that drone deliveries a...,not verified,under review,343296.000,19425.000,241.000,1.000,0.000
1,2,claim,4014381136,32,someone shared with me that there are more mic...,not verified,active,140877.000,77355.000,19034.000,1161.000,684.000
2,3,claim,9859838091,31,someone shared with me that american industria...,not verified,active,902185.000,97690.000,2858.000,833.000,329.000
3,4,claim,1866847991,25,someone shared with me that the metro of st. p...,not verified,active,437506.000,239954.000,34812.000,1234.000,584.000
4,5,claim,7105231098,19,someone shared with me that the number of busi...,not verified,active,56167.000,34987.000,4110.000,547.000,152.000
5,6,claim,8972200955,35,someone shared with me that gross domestic pro...,not verified,under review,336647.000,175546.000,62303.000,4293.000,1857.000
6,7,claim,4958886992,16,someone shared with me that elvis presley has ...,not verified,active,750345.000,486192.000,193911.000,8616.000,5446.000
7,8,claim,2270982263,41,someone shared with me that the best selling s...,not verified,active,547532.000,1072.000,50.000,22.000,11.000
8,9,claim,5235769692,50,someone shared with me that about half of the ...,not verified,active,24819.000,10160.000,1050.000,53.000,27.000
9,10,claim,4660861094,45,someone shared with me that it would take a 50...,verified,active,931587.000,171051.000,67739.000,4104.000,2540.000


In [5]:
# --- Schema & missingness: tidy table vs. df.info() text dump ---------------
# Build a small DataFrame summarizing each column's dtype and missingness.
# This renders cleanly in notebooks and is easier to scan/sort than raw text.

schema = (
    df.dtypes.rename("dtype").to_frame()             # column -> dtype
      .assign(
          non_null = df.notna().sum(),               # count of non-missing
          missing  = df.isna().sum(),                # count of missing
          missing_pct = lambda t: 100 * t["missing"] / len(df)  # % missing
      )
      .reset_index()
      .rename(columns={"index": "column"})
      .sort_values("column")
)

# Pretty formatting for percent column; keeps numeric types for others
display(schema.style.format({"missing_pct": "{:.2f}%"}))

,column,dtype,non_null,missing,missing_pct
0,#,int64,19382,0,0.00%
6,author_ban_status,object,19382,0,0.00%
1,claim_status,object,19084,298,1.54%
5,verified_status,object,19382,0,0.00%
11,video_comment_count,float64,19084,298,1.54%
10,video_download_count,float64,19084,298,1.54%
3,video_duration_sec,int64,19382,0,0.00%
2,video_id,int64,19382,0,0.00%
8,video_like_count,float64,19084,298,1.54%
9,video_share_count,float64,19084,298,1.54%


In [6]:
# --- Numeric summary: go beyond describe() with median, skew, missing% ------
# We start from describe(), transpose so rows = features, and add useful stats.

# Select numeric-only to avoid warnings in skew/describe
num_cols = df.select_dtypes(include="number")

# If there are no numeric columns, skip gracefully
if num_cols.shape[1] == 0:
    print("No numeric columns to summarize.")
else:
    num_summary = (
        num_cols.describe(percentiles=[0.25, 0.5, 0.75]).T
          .rename(columns={"50%": "median"})            # clearer than '50%'
          .assign(
              missing = num_cols.isna().sum(),          # missing counts
              missing_pct = lambda t: 100 * t["missing"] / len(df),
              skew = num_cols.skew(numeric_only=True)   # distribution asymmetry
          )
          # Order columns for stakeholder readability
          .loc[:, ["count","mean","std","min","25%","median","75%","max","skew","missing_pct"]]
          .sort_index()
    )

    # Format skew and missing% to two decimals; leave others as-is
    display(num_summary.style.format({"skew": "{:.2f}", "missing_pct": "{:.2f}%"}))

,count,mean,std,min,25%,median,75%,max,skew,missing_pct
#,19382.000000,9691.500000,5595.245794,1.000000,4846.250000,9691.500000,14536.750000,19382.000000,0.00,0.00%
video_comment_count,19084.000000,349.312146,799.638865,0.000000,1.000000,9.000000,292.000000,9599.000000,3.89,1.54%
video_download_count,19084.000000,1049.429627,2004.299894,0.000000,7.000000,46.000000,1156.250000,14994.000000,2.74,1.54%
video_duration_sec,19382.000000,32.421732,16.229967,5.000000,18.000000,32.000000,47.000000,60.000000,0.00,0.00%
video_id,19382.000000,5627454067.339129,2536440464.169367,1234959018.000000,3430416807.250000,5618663579.000000,7843960211.250000,9999873075.000000,0.00,0.00%
video_like_count,19084.000000,84304.636030,133420.546814,0.000000,810.750000,3403.500000,125020.000000,657830.000000,1.79,1.54%
video_share_count,19084.000000,16735.248323,32036.174350,0.000000,115.000000,717.000000,18222.000000,256130.000000,2.72,1.54%
video_view_count,19084.000000,254708.558688,322893.280814,20.000000,4942.500000,9954.500000,504327.000000,999817.000000,0.93,1.54%


In [7]:
# --- Target label distribution: class balance sanity check ------------------
# Replace 'claim_status' with your actual label column name if different.

label_col = "claim_status"

if label_col in df.columns:
    label_counts = (
        df[label_col]
          .value_counts(dropna=False)                      # include NaN if present
          .to_frame("count")
          .assign(pct=lambda s: (s["count"] / s["count"].sum() * 100).round(2))
    )
    display(label_counts)
else:
    print(f"Label column '{label_col}' not found; skipping class balance table.")

,count,pct
claim_status,,
claim,9608,49.570
opinion,9476,48.890
NaN,298,1.540


### Data Inspection Summary

- **Dataset structure:** The dataframe contains 19,382 observations of TikTok videos, each with categorical, text, and numerical fields describing claims, opinions, and metadata.  
- **Variable types:** Columns include five `float64`, three `int64`, and four `object` datatypes.  
- **Missing values:** Several variables have nulls, including *claim status*, *video transcription*, and the count variables.  
- **Distributional checks:** Many count variables show long right tails and potential outliers. Their maximum values are orders of magnitude larger than the interquartile ranges, resulting in high standard deviations.  

## Exploratory Data Analysis: Claims, Engagement, and Author Status

In [8]:
# Distribution of claim vs opinion labels
df['claim_status'].value_counts()

claim_status
claim      9608
opinion    9476
Name: count, dtype: int64

- The dataset is relatively balanced between *claim* and *opinion* labels.

In [9]:
# Drop unlabeled rows before modeling
df = df[df['claim_status'].notna()].copy()

- Note: A small number of rows lack a `claim_status` label. These observations are excluded to ensure clean, labeled data for modeling.

In [10]:
# Average and median view counts for claim vs opinion videos
claims = df[df['claim_status'] == 'claim']
opinions = df[df['claim_status'] == 'opinion']

print("Claims — mean:", claims['video_view_count'].mean(), 
      "median:", claims['video_view_count'].median())
print("Opinions — mean:", opinions['video_view_count'].mean(), 
      "median:", opinions['video_view_count'].median())

Claims — mean: 501029.4527477102 median: 501555.0
Opinions — mean: 4956.43224989447 median: 4953.0


- Within each label, mean and median view counts are close.  
- However, claim videos generally attract far higher view counts than opinion videos.

In [11]:
# Normalize author ban status labels for consistency
df['author_ban_status'] = (
    df['author_ban_status']
    .str.strip().str.lower()
    .replace({'under  review': 'under review'})  # collapse double spaces
)

# Cross-tabulation of claim status and author ban status
df.groupby(['claim_status', 'author_ban_status']).size().unstack(fill_value=0)

author_ban_status,active,banned,under review
claim_status,,,
claim,6566,1439,1603
opinion,8817,196,463


- Claim videos are more common among banned authors than opinion videos.  
- This could reflect stricter moderation for claims, but causation cannot be inferred:  
  - Claim content may inherently invite more scrutiny.  
  - Or authors posting claims may also post other content that violates terms.  
- Important: we cannot conclude that specific videos caused a ban.

In [12]:
# Engagement metrics (mean/median) by author ban status
df.groupby('author_ban_status').agg({
    'video_view_count': ['mean', 'median'],
    'video_like_count': ['mean', 'median'],
    'video_share_count': ['mean', 'median']
})

video_view_count            video_like_count             \
                              mean     median             mean     median   
author_ban_status                                                           
active                  215927.040   8616.000        71036.534   2222.000   
banned                  445845.439 448201.000       153017.237 105573.000   
under review            392204.836 365245.500       128718.050  71204.500   

                  video_share_count            
                               mean    median  
author_ban_status                              
active                    14111.466   437.000  
banned                    29998.943 14468.000  
under review              25774.697  9444.000

- Banned authors’ videos tend to have substantially higher engagement (views, likes, shares).  
- Mean values are consistently greater than medians, pointing to long-tailed distributions.  

In [13]:
# Engagement metrics (count, mean, median) by author ban status
df.groupby('author_ban_status').agg({
    'video_view_count': ['count', 'mean', 'median'],
    'video_like_count': ['count', 'mean', 'median'],
    'video_share_count': ['count', 'mean', 'median']
})

video_view_count                       video_like_count  \
                             count       mean     median            count   
author_ban_status                                                           
active                       15383 215927.040   8616.000            15383   
banned                        1635 445845.439 448201.000             1635   
under review                  2066 392204.836 365245.500             2066   

                                        video_share_count                      
                        mean     median             count      mean    median  
author_ban_status                                                              
active             71036.534   2222.000             15383 14111.466   437.000  
banned            153017.237 105573.000              1635 29998.943 14468.000  
under review      128718.050  71204.500              2066 25774.697  9444.000

- Banned and under-review authors receive far more engagement than active authors.  
- Count columns provide context: sample sizes differ between groups.  
- Means >> medians confirm outliers driving up averages.  

In [14]:
# Median share counts by author ban status
df.groupby('author_ban_status')['video_share_count'].median()

author_ban_status
active           437.000
banned         14468.000
under review    9444.000
Name: video_share_count, dtype: float64

- Banned authors’ videos have a median share count ~33× higher than active authors.  
- This dramatic difference highlights the role of virality and moderation bias.  

In [15]:
# Per-view engagement ratios (safe against zero views)
denom = df['video_view_count'].replace(0, np.nan)

df['likes_per_view']    = df['video_like_count']    / denom
df['comments_per_view'] = df['video_comment_count'] / denom
df['shares_per_view']   = df['video_share_count']   / denom

# Replace NaNs from divide-by-zero with 0
df[['likes_per_view','comments_per_view','shares_per_view']] = (
    df[['likes_per_view','comments_per_view','shares_per_view']].fillna(0)
)

# Summary stats by claim and author ban status
df.groupby(['claim_status', 'author_ban_status']).agg({
    'likes_per_view': ['mean', 'median'],
    'comments_per_view': ['mean', 'median'],
    'shares_per_view': ['mean', 'median']
})

likes_per_view        comments_per_view         \
                                         mean median              mean median   
claim_status author_ban_status                                                  
claim        active                     0.330  0.327             0.001  0.001   
             banned                     0.345  0.359             0.001  0.001   
             under review               0.328  0.321             0.001  0.001   
opinion      active                     0.220  0.218             0.001  0.000   
             banned                     0.207  0.198             0.000  0.000   
             under review               0.226  0.228             0.001  0.000   

                               shares_per_view         
                                          mean median  
claim_status author_ban_status                         
claim        active                      0.065  0.049  
             banned                      0.068  0.052  
             under review                0.066  0.050  
opinion      active                      0.044  0.032  
             banned                      0.041  0.031  
             under review                0.044  0.035

- Claim videos show higher per-view engagement rates than opinion videos.  
- Within claims, banned authors’ videos have slightly higher rates than others.  
- Within opinions, active/under-review authors outperform banned authors.  

In [16]:
# Sanity check: ensure no NaN/inf values remain in per-view ratios
bad = df[['likes_per_view','comments_per_view','shares_per_view']].replace([np.inf,-np.inf], np.nan)
assert bad.isna().sum().sum() == 0, "Found NaN/inf in per-view ratios"
print("Per-view ratios look clean ✅")

Per-view ratios look clean ✅


In [17]:
# Per-view engagement ratios with counts (sample size context)
df.groupby(['claim_status', 'author_ban_status']).agg({
    'likes_per_view': ['count', 'mean', 'median'],
    'comments_per_view': ['count', 'mean', 'median'],
    'shares_per_view': ['count', 'mean', 'median']
})

likes_per_view              comments_per_view  \
                                        count  mean median             count   
claim_status author_ban_status                                                 
claim        active                      6566 0.330  0.327              6566   
             banned                      1439 0.345  0.359              1439   
             under review                1603 0.328  0.321              1603   
opinion      active                      8817 0.220  0.218              8817   
             banned                       196 0.207  0.198               196   
             under review                 463 0.226  0.228               463   

                                            shares_per_view               
                                mean median           count  mean median  
claim_status author_ban_status                                            
claim        active            0.001  0.001            6566 0.065  0.049  
             banned            0.001  0.001            1439 0.068  0.052  
             under review      0.001  0.001            1603 0.066  0.050  
opinion      active            0.001  0.000            8817 0.044  0.032  
             banned            0.000  0.000             196 0.041  0.031  
             under review      0.001  0.000             463 0.044  0.035

- Adding counts confirms robust sample sizes for both claims and opinions.  
- Results validate earlier findings: claim status drives engagement more than ban status.  
- Again, right-skew is visible (mean > median).

## Section 2 Summary: Exploratory Analysis

- **Label balance:** The dataset is reasonably balanced between claims and opinions, making it suitable for supervised modeling.  
- **Claim dynamics:** Claim videos generally attract higher view counts and engagement than opinion videos.  
- **Author ban patterns:** Banned or under-review authors account for disproportionately higher engagement, though this reflects correlation only and causality cannot be inferred from the dataset.  
- **Engagement distributions:** Across groups, means are consistently greater than medians, confirming right-skewed, long-tailed distributions.  
- **Shares:** Median share counts are substantially higher for banned authors, suggesting virality dynamics that warrant closer review.  
- **Normalized engagement:** Per-view rates (e.g., likes per view, comments per view, shares per view) highlight claim status as the stronger predictor of engagement compared to ban status.  
- **Sample size context:** Including counts confirms that observed trends are based on robust subsets of the data, not artifacts of small groups.  

**Overall:** Claim content and author ban status both correlate with higher engagement, though claim status appears to drive engagement more strongly. These findings will inform subsequent feature engineering and model development.  